In [45]:
%%html
<style>

.dataframe td {
    white-space: normal !important;       /* consente il wrapping */
    word-wrap: break-word;               /* forza il wrapping se serve */
    max-width: 900px;                    /* larghezza massima di una cella */
}
</style>

In [2]:
%load_ext autoreload
%autoreload 2

import os
os.environ['PATH'] = './ffmpeg-7.0.2-amd64-static:' + os.environ['PATH']

from time import time
from datetime import datetime
import random
import torch
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 1500)
pd.set_option('display.max_columns', 1500)
#pd.set_option('display.max_seq_item_length', 500)
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_info_columns', 500)
pd.set_option('display.max_info_rows', 500)
pd.set_option('display.expand_frame_repr', True)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.width', 21000)

pd.set_option('display.date_dayfirst', True)


import matplotlib.pyplot as plt
from PIL import Image
from IPython.display import HTML
import matplotlib.animation as animation
from matplotlib.animation import FFMpegWriter
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128

from arguments import prepare_finetuning_args, Args
from dataset import build_dataset
from torch.utils.data import DataLoader
from engine_for_finetuning import validation_one_epoch
import models
from timm.models import create_model

from dataset.build_dataset import calc_tile_offsets, create_df_video_from_master_df, solve_paths, get_train_test_validation_df
from view_test_tiles import display_video_clip, save_frames_parallel, make_animation_parallel_ffmpeg, filling_missing_tile

from model_analysis import predict_label, get_path_pred_label, create_df_predictions, get_only_labels
from dataset.data_manager import DataManager, BuildDataset

input_dir = "../fromgcloud"
output_dir = "../airmassRGB/supervised/"


def make_validation_data_builder_from_manos_tracks(manos_track_file, input_dir, output_dir):
    args = prepare_finetuning_args()

    output_dir = solve_paths(output_dir)
    input_dir = solve_paths(input_dir)

    tracks_df = pd.read_csv(manos_track_file, parse_dates=['time', 'start_time', 'end_time'])
    tracks_df_train, tracks_df_test, tracks_df_val = get_train_test_validation_df(tracks_df, 0.7, 0.15)
    val_b = BuildDataset(type='SUPERVISED', args=args)
    val_b.get_data_ready(tracks_df_val, input_dir, output_dir, csv_file="val_manos_w", is_to_balance=False)
    #val_b.get_data_ready(tracks_df_test, input_dir, output_dir, csv_file="val_manos_w", is_to_balance=True)
    return val_b

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Impossibile caricare la libreria torch dynamo


In [3]:
args = prepare_finetuning_args()
args.device='cpu'

In [4]:
###### carico il dataset
val_m = DataManager(is_train=False, args=args, specify_data_path=args.val_path)
val_m.create_classif_dataloader(args)

Getting dataset...
DATASET length: 2400
Creo il DistributedSampler con world_size 1 e rank 0
Sampler_train = <torch.utils.data.distributed.DistributedSampler object at 0x7f5178bf27c0>
Batch_size local: 1


In [ ]:

# voglio prendere le predizioni
get_prediction = False
args.test_mode = True

# istanzia l'oggetto del modello 
print(f"Creating model: {args.model} (nb_classes={args.nb_classes})")

args.init_ckpt = './output/checkpoint-best-90_25lug25.pth'

model = create_model(
    args.model,
    num_classes=args.nb_classes,
    drop_rate=0.0,
    drop_path_rate=args.drop_path,
    #attn_drop_rate=0.0,
    drop_block_rate=None,
    **args.__dict__
)
model.to(args.device)
model.eval()  

In [4]:
#all_paths, all_preds, all_labels = get_path_pred_label(model, val_m.data_loader)
all_paths, all_preds, all_labels = get_only_labels(val_m.data_loader)
all_labels_array = np.array(all_labels)
all_preds_array = np.array(all_preds)
df_predictions = create_df_predictions(all_paths, all_preds, all_labels)

In [8]:
df_predictions = pd.read_csv('df_predictions.csv') #, parse_dates=['time', 'start_time', 'end_time'])
df_predictions

,path,predictions,labels
0,08-11-2021_2100_213_0,0,0
1,08-11-2021_1400_0_196,0,0
2,07-09-2023_0800_426_196,0,0
3,02-03-2023_0000_852_196,0,0
4,08-09-2023_0000_852_196,1,0
...,...,...,...
2395,01-03-2023_1100_639_196,0,0
2396,06-09-2023_0200_0_196,0,0
2397,07-09-2023_0400_0_0,0,0
2398,06-09-2023_2000_852_196,0,0


In [15]:
y_true = df_predictions['labels'].values
y_pred = df_predictions['predictions'].values

tp = ((y_pred == 1) & (y_true == 1)).sum().item()
tn = ((y_pred == 0) & (y_true == 0)).sum().item()
fp = ((y_pred == 1) & (y_true == 0)).sum().item()
fn = ((y_pred == 0) & (y_true == 1)).sum().item()

fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
fnr = fn / (fn + tp) if (fn + tp) > 0 else 0

In [21]:
#fpr, fnr
(y_pred==y_true).sum()/ len(y_true)

0.8325

In [5]:
#create_df_video_from_master_df
val_builder = make_validation_data_builder_from_manos_tracks("medicanes_new_windows.csv", input_dir, output_dir)

Cicloni nel train: 12, cicloni nel test: 3, cicloni nella validation: 3
Train rows: 564, Test rows: 176, Validation rows: 206
23 days 08:45:00
7 days 08:05:00
8 days 13:25:00
sorted_metadata_files num :  2437
1)  ->
Con cicloni: 18
Senza cicloni: 198

Creazione delle folder per i 216 video...	Salvati 0 file - Erano già presenti 3456 file - File totali 3456
216 video per il periodo (effettivo) da 2021-11-08 11:45:00 a 2021-11-09 06:00:00

2)  ->
Con cicloni: 66
Senza cicloni: 714

Creazione delle folder per i 780 video...	Salvati 0 file - Erano già presenti 12480 file - File totali 12480
780 video per il periodo (effettivo) da 2023-02-27 19:45:00 a 2023-03-02 13:00:00

3)  ->
Con cicloni: 117
Senza cicloni: 1287

Creazione delle folder per i 1404 video...	Salvati 0 file - Erano già presenti 22464 file - File totali 22464
1404 video per il periodo (effettivo) da 2023-09-05 04:45:00 a 2023-09-10 02:00:00



In [9]:
# ottengo le predictions associate ai video con i rispettivi path
df_filtrato_on_video_path = val_builder.df_video.merge(df_predictions, on='path')

In [50]:
#all(df_filtrato_on_video_path.label == df_filtrato_on_video_path.labels)   # ok
#df_filtrato_on_video_path.label.sum()  # ok
df_filtrato_on_video_path.head(130)  #[['path','orig_paths', 'label']]

# 8-11-2021_1800_426_0  video con label 1 corretta

,video_id,tile_offset_x,tile_offset_y,path,label,start_time,end_time,orig_paths,predictions,labels
0,0,0,0,08-11-2021_1300_0_0,0,2021-11-08 11:45:00,2021-11-08 13:00:00,"[../fromgcloud/airmass_rgb_20211108_1145.png, ../fromgcloud/airmass_rgb_20211108_1150.png, ../fromgcloud/airmass_rgb_20211108_1155.png, ../fromgcloud/airmass_rgb_20211108_1200.png, ../fromgcloud/airmass_rgb_20211108_1205.png, ../fromgcloud/airmass_rgb_20211108_1210.png, ../fromgcloud/airmass_rgb_20211108_1215.png, ../fromgcloud/airmass_rgb_20211108_1220.png, ../fromgcloud/airmass_rgb_20211108_1225.png, ../fromgcloud/airmass_rgb_20211108_1230.png, ../fromgcloud/airmass_rgb_20211108_1235.png, ../fromgcloud/airmass_rgb_20211108_1240.png, ../fromgcloud/airmass_rgb_20211108_1245.png, ../fromgcloud/airmass_rgb_20211108_1250.png, ../fromgcloud/airmass_rgb_20211108_1255.png, ../fromgcloud/airmass_rgb_20211108_1300.png]",0,0
1,1,0,0,08-11-2021_1400_0_0,0,2021-11-08 12:45:00,2021-11-08 14:00:00,"[../fromgcloud/airmass_rgb_20211108_1245.png, ../fromgcloud/airmass_rgb_20211108_1250.png, ../fromgcloud/airmass_rgb_20211108_1255.png, ../fromgcloud/airmass_rgb_20211108_1300.png, ../fromgcloud/airmass_rgb_20211108_1305.png, ../fromgcloud/airmass_rgb_20211108_1310.png, ../fromgcloud/airmass_rgb_20211108_1315.png, ../fromgcloud/airmass_rgb_20211108_1320.png, ../fromgcloud/airmass_rgb_20211108_1325.png, ../fromgcloud/airmass_rgb_20211108_1330.png, ../fromgcloud/airmass_rgb_20211108_1335.png, ../fromgcloud/airmass_rgb_20211108_1340.png, ../fromgcloud/airmass_rgb_20211108_1345.png, ../fromgcloud/airmass_rgb_20211108_1350.png, ../fromgcloud/airmass_rgb_20211108_1355.png, ../fromgcloud/airmass_rgb_20211108_1400.png]",0,0
2,2,0,0,08-11-2021_1500_0_0,0,2021-11-08 13:45:00,2021-11-08 15:00:00,"[../fromgcloud/airmass_rgb_20211108_1345.png, ../fromgcloud/airmass_rgb_20211108_1350.png, ../fromgcloud/airmass_rgb_20211108_1355.png, ../fromgcloud/airmass_rgb_20211108_1400.png, ../fromgcloud/airmass_rgb_20211108_1405.png, ../fromgcloud/airmass_rgb_20211108_1410.png, ../fromgcloud/airmass_rgb_20211108_1415.png, ../fromgcloud/airmass_rgb_20211108_1420.png, ../fromgcloud/airmass_rgb_20211108_1425.png, ../fromgcloud/airmass_rgb_20211108_1430.png, ../fromgcloud/airmass_rgb_20211108_1435.png, ../fromgcloud/airmass_rgb_20211108_1440.png, ../fromgcloud/airmass_rgb_20211108_1445.png, ../fromgcloud/airmass_rgb_20211108_1450.png, ../fromgcloud/airmass_rgb_20211108_1455.png, ../fromgcloud/airmass_rgb_20211108_1500.png]",0,0
3,3,0,0,08-11-2021_1600_0_0,0,2021-11-08 14:45:00,2021-11-08 16:00:00,"[../fromgcloud/airmass_rgb_20211108_1445.png, ../fromgcloud/airmass_rgb_20211108_1450.png, ../fromgcloud/airmass_rgb_20211108_1455.png, ../fromgcloud/airmass_rgb_20211108_1500.png, ../fromgcloud/airmass_rgb_20211108_1505.png, ../fromgcloud/airmass_rgb_20211108_1510.png, ../fromgcloud/airmass_rgb_20211108_1515.png, ../fromgcloud/airmass_rgb_20211108_1520.png, ../fromgcloud/airmass_rgb_20211108_1525.png, ../fromgcloud/airmass_rgb_20211108_1530.png, ../fromgcloud/airmass_rgb_20211108_1535.png, ../fromgcloud/airmass_rgb_20211108_1540.png, ../fromgcloud/airmass_rgb_20211108_1545.png, ../fromgcloud/airmass_rgb_20211108_1550.png, ../fromgcloud/airmass_rgb_20211108_1555.png, ../fromgcloud/airmass_rgb_20211108_1600.png]",0,0
4,4,0,0,08-11-2021_1700_0_0,0,2021-11-08 15:45:00,2021-11-08 17:00:00,"[../fromgcloud/airmass_rgb_20211108_1545.png, ../fromgcloud/airmass_rgb_20211108_1550.png, ../fromgcloud/airmass_rgb_20211108_1555.png, ../fromgcloud/airmass_rgb_20211108_1600.png, ../fromgcloud/airmass_rgb_20211108_1605.png, ../fromgcloud/airmass_rgb_20211108_1610.png, ../fromgcloud/airmass_rgb_20211108_1615.png, ../fromgcloud/airmass_rgb_20211108_1620.png, ../fromgcloud/airmass_rgb_20211108_1625.png, ../fromgcloud/airmass_rgb_20211108_1630.png, ../fromgcloud/airmass_rgb_20211108_1635.png, ../fromgcloud/airmass_rgb_20211108_1640.png, ../fromgcloud/airmass_rgb_20211108_1645.png, ../fromgcloud/airmass_rgb_20211108_1650.png, ../fromgclo

In [10]:
# Espando dataframe con le associazioni path_offsets -> predictions
records = []
for _, row in df_filtrato_on_video_path.iterrows():
    for orig_path in row['orig_paths']:
        records.append({
            'path': orig_path,
            'predictions': row['predictions'],
            'tmp_label': row['labels'],
            'tile_offset_x': row['tile_offset_x'],
            'tile_offset_y': row['tile_offset_y']
        })

# Li trasformiamo in un nuovo DataFrame
df_mapping = pd.DataFrame(records)
print(df_mapping[['path', 'tile_offset_x', 'tile_offset_y','tmp_label']].duplicated().sum())
#Non ci sono path duplicati se in combinazione con gli offsets, 
# a parte se i video sono generati con un certo overlap in tempo!

# ottengo le predictions per ogni immagine
df_mapping.drop_duplicates(inplace=True)
df_mapping.shape


9472


(29200, 5)

In [55]:
df_mapping.iloc[1000:1200]   #8-11-2021_1800_426_0

,path,predictions,tmp_label,tile_offset_x,tile_offset_y
1308,../fromgcloud/airmass_rgb_20211108_2145.png,0,1,426,0
1309,../fromgcloud/airmass_rgb_20211108_2150.png,0,1,426,0
1310,../fromgcloud/airmass_rgb_20211108_2155.png,0,1,426,0
1311,../fromgcloud/airmass_rgb_20211108_2200.png,0,1,426,0
1316,../fromgcloud/airmass_rgb_20211108_2205.png,0,1,426,0
1317,../fromgcloud/airmass_rgb_20211108_2210.png,0,1,426,0
1318,../fromgcloud/airmass_rgb_20211108_2215.png,0,1,426,0
1319,../fromgcloud/airmass_rgb_20211108_2220.png,0,1,426,0
1320,../fromgcloud/airmass_rgb_20211108_2225.png,0,1,426,0
1321,../fromgcloud/airmass_rgb_20211108_2230.png,0,1,426,0


In [11]:
# ottengo il dataframe per creare i video , mi servono anche le 'x_pix', 'y_pix', 'source'
df_data_merg = df_mapping.merge(val_builder.master_df, on=['path', 'tile_offset_x', 'tile_offset_y'], how='left').drop(columns='label').rename(columns={'tmp_label':'label'})

In [60]:
### devo risolvere con le tile mancanti
offsets = calc_tile_offsets(stride_x=213, stride_y=196)
df_offsets = pd.DataFrame(offsets, columns=['tile_offset_x', 'tile_offset_y'])
def expand_group(group):
    merged = df_offsets.merge(group, on=['tile_offset_x', 'tile_offset_y'], how='left', indicator=True)
    path_value = group['path'].iloc[0]
    merged['path'] = path_value

    extra_cols = [col for col in group.columns if col not in ['path', 'tile_offset_x', 'tile_offset_y']]
    # Ricopia i valori costanti del gruppo originale
    for col in ['datetime']:
        val = group[col].iloc[0]   #.mode()[0] if not group[col].isnull().all() else None
        merged[col] = merged[col].fillna(val)

    merged[filling_missing_tile] = merged['_merge'] == 'left_only'  # True se mancava
    return merged[['path', 'tile_offset_x', 'tile_offset_y'] + extra_cols + [filling_missing_tile]]

# Applichiamo la funzione a ogni path
expanded_df = df_data_merg.groupby('path', group_keys=False).apply(expand_group).reset_index(drop=True)
expanded_df.predictions = expanded_df.predictions.astype('Int8')
expanded_df.label = expanded_df.label.astype('Int8')

/tmp/ipykernel_1912134/4063762423.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  expanded_df = df_data_merg.groupby('path', group_keys=False).apply(expand_group).reset_index(drop=True)


In [57]:
df_data_merg.iloc[1000:1200]

,path,predictions,label,tile_offset_x,tile_offset_y,datetime,x_pix,y_pix,source,id_cyc_unico,start_time,end_time
1000,../fromgcloud/airmass_rgb_20211108_2145.png,0,1,426,0,2021-11-08 21:45:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1001,../fromgcloud/airmass_rgb_20211108_2150.png,0,1,426,0,2021-11-08 21:50:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1002,../fromgcloud/airmass_rgb_20211108_2155.png,0,1,426,0,2021-11-08 21:55:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1003,../fromgcloud/airmass_rgb_20211108_2200.png,0,1,426,0,2021-11-08 22:00:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1004,../fromgcloud/airmass_rgb_20211108_2205.png,0,1,426,0,2021-11-08 22:05:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1005,../fromgcloud/airmass_rgb_20211108_2210.png,0,1,426,0,2021-11-08 22:10:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1006,../fromgcloud/airmass_rgb_20211108_2215.png,0,1,426,0,2021-11-08 22:15:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1007,../fromgcloud/airmass_rgb_20211108_2220.png,0,1,426,0,2021-11-08 22:20:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1008,../fromgcloud/airmass_rgb_20211108_2225.png,0,1,426,0,2021-11-08 22:25:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00
1009,../fromgcloud/airmass_rgb_20211108_2230.png,0,1,426,0,2021-11-08 22:30:00,[444],[181],['ERA5'],0,2021-11-08 10:20:00,2021-11-09 06:35:00


In [13]:
make_animation_parallel_ffmpeg(df_data_merg, nomefile='validation_b1')


>>> Generazione dei frame PNG in anim_frames_validation_b1
 abbiamo 2410 gruppi
Tutti i frame sono stati generati
19.29 minuti

>>> Creazione del video MP4 con ffmpeg...


ffmpeg version 7.0.2-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      59.  8.100 / 59.  8.100
  libavcodec     61.  3.100 / 61.  3.100
  libavformat   


Video salvato: validation_b1.mp4

Cartella anim_frames_validation_b1 già esistente, non ricreo i frame. Controlla se il video è già stato creato.
Video già esistente: validation_b1.mp4


[out#0/mp4 @ 0x6c6cf00] video:115945KiB audio:0KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 0.035160%
frame= 2411 fps= 53 q=-1.0 Lsize=  115986KiB time=00:04:00.84 bitrate=3945.2kbits/s speed=5.25x    
[libx264 @ 0x6c6ee80] frame I:11    Avg QP:15.11  size:202265
[libx264 @ 0x6c6ee80] frame P:1483  Avg QP:17.48  size: 64593
[libx264 @ 0x6c6ee80] frame B:917   Avg QP:20.02  size: 22585
[libx264 @ 0x6c6ee80] consecutive B-frames: 48.8%  1.5%  0.1% 49.6%
[libx264 @ 0x6c6ee80] mb I  I16..4:  6.0% 21.9% 72.1%
[libx264 @ 0x6c6ee80] mb P  I16..4:  0.1%  1.0%  1.2%  P16..4: 32.2% 29.4% 30.5%  0.0%  0.0%    skip: 5.7%
[libx264 @ 0x6c6ee80] mb B  I16..4:  0.0%  0.0%  0.2%  B16..8: 22.0% 10.9%  6.4%  direct:17.4%  skip:43.1%  L0:18.7% L1:35.2% BI:46.1%
[libx264 @ 0x6c6ee80] 8x8 transform intra:36.2% inter:44.0%
[libx264 @ 0x6c6ee80] coded y,uvDC,uvAC intra: 89.1% 97.6% 96.6% inter: 55.2% 65.6% 45.0%
[libx264 @ 0x6c6ee80] i16 v,h,dc,p: 36% 32%  7% 25%
[libx264 @ 0x6c6e

In [58]:
grouped = df_data_merg.groupby("path", dropna=False)
print(f" abbiamo {len(list(grouped))} gruppi", flush=True)

 abbiamo 2410 gruppi


In [59]:
list_grouped_df = list(grouped)

In [64]:
list_grouped_df[4]

('../fromgcloud/airmass_rgb_20211108_1205.png',
                          path                      predictions  label  tile_offset_x  tile_offset_y       datetime      x_pix  y_pix    source   id_cyc_unico      start_time           end_time     
 4     ../fromgcloud/airmass_rgb_20211108_1205.png       0         0           0              0      2021-11-08 12:05:00     []     []        []        0                      NaT                 NaT
 224   ../fromgcloud/airmass_rgb_20211108_1205.png       0         0           0            196      2021-11-08 12:05:00     []     []        []        0                      NaT                 NaT
 444   ../fromgcloud/airmass_rgb_20211108_1205.png       0         0         213              0      2021-11-08 12:05:00     []     []        []        0                      NaT                 NaT
 664   ../fromgcloud/airmass_rgb_20211108_1205.png       0         0         213            196      2021-11-08 12:05:00     []     []        []        0   

In [74]:
from view_test_tiles import render_and_save_frame

render_and_save_frame((5, list_grouped_df, "./"), overwrite=True)

'./airmass_rgb_20211108_1210.png'